In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from datetime import datetime

# =========================
# 0. 로딩 로그
# =========================
def loading(msg):
    print(f"[LOADING] {msg}")

def now():
    return datetime.now().strftime("%H:%M:%S")

# =========================
# 1. Selenium 설정
# =========================
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 25)

loading("브라우저 실행")

# =========================
# 2. 무신사 접속 (브랜드 탭)
# =========================
driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=M&tabId=brand")
time.sleep(5)
loading("무신사 접속 완료")

# =========================
# 3. Joyride 툴팁 제거 (최대 2개)
# =========================
def close_joyride_tooltip():
    try:
        btns = driver.find_elements(
            By.CSS_SELECTOR,
            'button[aria-label="닫기"], button[data-action="primary"]'
        )
        for btn in btns:
            try:
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(0.4)
            except:
                continue
    except:
        pass

close_joyride_tooltip()
time.sleep(1)
close_joyride_tooltip()

# =========================
# 4. 브랜드 메뉴 탭 클릭 (CSS 기반)
# =========================
brand_tabs = wait.until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, 'span[data-mds="TabTextItem"]')
    )
)

for tab in brand_tabs:
    if tab.text.strip().startswith("브랜드"):
        driver.execute_script("arguments[0].click();", tab)
        loading("브랜드 메뉴 탭 클릭")
        break

time.sleep(2)

# =========================
# 5. 브랜드 메뉴 스크롤 컨테이너
# =========================
scroll_container = wait.until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'div[data-scroll-id="menu-sub-area"]')
    )
)
loading("브랜드 메뉴 로딩 완료")

# =========================
# 6. 패션소품 버튼 클릭
# =========================
fashion_btn = wait.until(
    EC.element_to_be_clickable((
        By.XPATH,
        '//div[@data-button-name="패션소품"]'
    ))
)
driver.execute_script("arguments[0].click();", fashion_btn)
loading("패션소품 카테고리 선택")
time.sleep(2)

# =========================
# 7. 브랜드 수집 (LIMIT 없음 / 끝까지)
# =========================
brand_data = {}

loading("브랜드 전체 수집 시작")

while True:
    brand_items = driver.find_elements(
        By.CSS_SELECTOR,
        'a[href*="/brand/"] div.UIBrandItem__NameWrap-sc-1un5tti-4'
    )

    before_cnt = len(brand_data)

    for name_wrap in brand_items:
        try:
            brand_link = name_wrap.find_element(By.XPATH, './ancestor::a')
            brand_url = brand_link.get_attribute("href")

            brand_kr = name_wrap.find_element(
                By.CSS_SELECTOR,
                'span.UIBrandItem__Name-sc-1un5tti-6'
            ).text.strip()

            try:
                brand_en = name_wrap.find_element(
                    By.CSS_SELECTOR,
                    'span.UIBrandItem__EnglishName-sc-1un5tti-7'
                ).text.strip()
            except:
                brand_en = None

            if brand_url and brand_url not in brand_data:
                brand_data[brand_url] = {
                    "brand_kr": brand_kr,
                    "brand_en": brand_en,
                    "brand_url": brand_url
                }

                # ✅ 인덱스 + 시간 로그
                print(f"[{now()}] index={len(brand_data)} | {brand_kr} | {brand_url}")

        except:
            continue

    # 🔥 더 이상 추가되는 브랜드가 없으면 종료
    if len(brand_data) == before_cnt:
        loading("더 이상 신규 브랜드 없음 → 전체 수집 종료")
        break

    # 🔥 virtual list 스크롤
    driver.execute_script(
        "arguments[0].scrollTop += 120;",
        scroll_container
    )
    time.sleep(0.6)

# =========================
# 8. 결과 정리
# =========================
df = pd.DataFrame(brand_data.values())

loading(f"전체 수집 완료 (총 {len(df)}개)")
print(df.head())
print(df.tail())

df.to_csv(
    "musinsa_fashion_goods_brands_all.csv",
    index=False,
    encoding="utf-8-sig"
)

loading("CSV 저장 완료")
